In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
df=pd.read_csv("../input/final_train_5.csv")
df=df.drop("Unnamed: 0",1)
df.head()

,SaleDollarCnt,PropertyID,censusblockgroup,Usecode,BedroomCnt,BathroomCnt,FinishedSquareFeet,GarageSquareFeet,LotSizeSquareFeet,StoryCnt,BuiltYear,ViewType,Latitude,Longitude,BGMedHomeValue,BGMedRent,BGMedYearBuilt,BGPctOwn,BGPctVacant,BGMedIncome,BGPctKids,BGMedAge,TransDate_4,TransDate_5,TransDate_6,TransDate_7,TransDate_8,TransDate_9,ZoneCodeCounty_A10,ZoneCodeCounty_A35,ZoneCodeCounty_AI1,ZoneCodeCounty_BO,ZoneCodeCounty_C1,ZoneCodeCounty_C140,ZoneCodeCounty_CB,ZoneCodeCounty_CBSO,ZoneCodeCounty_CM2,ZoneCodeCounty_DC,ZoneCodeCounty_DCE,ZoneCodeCounty_DUC,...,ZoneCodeCounty_RS9600,ZoneCodeCounty_RSA 4,ZoneCodeCounty_RSA 6,ZoneCodeCounty_RSA 8,ZoneCodeCounty_RSE,ZoneCodeCounty_RSX 35,ZoneCodeCounty_RSX 7.2,ZoneCodeCounty_RSX 8.5,ZoneCodeCounty_SF 5000,ZoneCodeCounty_SF 7200,ZoneCodeCounty_SF 9600,ZoneCodeCounty_SFD,ZoneCodeCounty_SFE,ZoneCodeCounty_SFR 10.0,ZoneCodeCounty_SFS,ZoneCodeCounty_SFSL,ZoneCodeCounty_SR1,ZoneCodeCounty_SR3,ZoneCodeCounty_SR30,ZoneCodeCounty_SR4.5,ZoneCodeCounty_SR6,ZoneCodeCounty_SR8,ZoneCodeCounty_SVV,ZoneCodeCounty_T,ZoneCodeCounty_TC,ZoneCodeCounty_TC4,ZoneCodeCounty_UL15000,ZoneCodeCounty_UL7200,ZoneCodeCounty_UL9600,ZoneCodeCounty_UM2400,ZoneCodeCounty_UR,ZoneCodeCounty_URPSO,ZoneCodeCounty_UV,ZoneCodeCounty_UVEV,ZoneCodeCounty_WD II,Missing ViewType,Missing GarageSquareFeet,Missing BGMedYearBuilt,Missing BGMedRent,Missing BGMedHomeValue
0,285000.0,-0.397302,0.0,0.0,0.633285,-0.375478,-0.320119,-0.049818,-0.197314,-1.012896,-0.371555,-0.326327,-1.610121,-0.100093,-1.829768,-0.992881,0.092411,-0.403853,0.462342,-1.433279,-1.193404,1.312359,-0.410486,2.115743,-0.447584,-0.494249,-0.441179,-0.415211,-0.037184,-0.033513,-0.00929,-0.00929,-0.022761,-0.013139,-0.016092,-0.00929,-0.00929,-0.00929,-0.00929,-0.013139,...,-0.034779,-0.044596,-0.138468,-0.037184,-0.00929,-0.018582,-0.102723,-0.00929,-0.48992,-0.204463,-0.055824,-0.026284,-0.024585,-0.00929,-0.068424,-0.053441,-0.022761,-0.050089,-0.00929,-0.084937,-0.177515,-0.052622,-0.00929,-0.00929,-0.016092,-0.00929,-0.016092,-0.085451,-0.013139,-0.013139,-0.062438,-0.087976,-0.077958,-0.058854,-0.013139,0.542108,-0.569910,-0.147578,-0.541975,-0.022761
1,309950.0,-0.397299,0.0,0.0,-0.521924,-0.375478,-0.031915,-0.775699,-0.041773,-1.012896,-0.799322,-0.361972,-0.481783,-0.315709,-1.416003,-0.787479,-0.244938,-0.878712,-0.539812,-1.125734,0.083579,0.420317,-0.410486,-0.472647,-0.447584,-0.494249,2.266652,-0.415211,-0.037184,-0.033513,-0.00929,-0.00929,-0.022761,-0.013139,-0.016092,-0.00929,-0.00929,-0.00929,-0.00929,-0.013139,...,-0.034779,-0.044596,-0.138468,-0.037184,-0.00929,-0.018582,-0.102723,-0.00929,-0.48992,-0.204463,-0.055824,-0.026284,-0.024585,-0.00929,-0.068424,-0.053441,-0.022761,-0.050089,-0.00929,-0.084937,-0.177515,-0.052622,-0.00929,-0.00929,-0.016092,-0.00929,-0.016092,-0.085451,-0.013139,-0.013139,-0.062438,-0.087976,-0.077958,-0.058854,-0.013139,-1.844651,-0.569910,-0.147578,-0.541975,-0.022761
2,476000.0,-0.397296,0.0,0.0,0.633285,-1.521526,-0.053264,0.449225,-0.220023,-1.012896,-0.728028,-0.374653,0.075045,-0.766053,-0.502016,-1.274357,-1.538110,-0.584218,-0.867046,-1.049419,-0.280155,0.137837,-0.410486,-0.472647,-0.447584,-0.494249,2.266652,-0.415211,-0.037184,-0.033513,-0.00929,-0.00929,-0.022761,-0.013139,-0.016092,-0.00929,-0.00929,-0.00929,-0.00929,-0.013139,...,-0.034779,-0.044596,-0.138468,-0.037184,-0.00929,-0.018582,-0.102723,-0.00929,-0.48992,4.890852,-0.055824,-0.026284,-0.024585,-0.00929,-0.068424,-0.053441,-0.022761,-0.050089,-0.00929,-0.084937,-0.177515,-0.052622,-0.00929,-0.00929,-0.016092,-0.00929,-0.016092,-0.085451,-0.013139,-0.013139,-0.062438,-0.087976,-0.077958,-0.058854,-0.013139,0.542108,-0.569910,-0.147578,-0.541975,-0.022761
3,324950.0,-0.397295,0.0,0.0,0.633285,-0.088966,0.384379,-0.514406,-0.005720,-1.012896,-0.478497,-0.361972,-1.142897,-0.563556,-0.839428,-0.850874,0.204861,-1.030035,0.109543,-1.396183,-0.171960,0.033766,-0.410486,-0.472647,-0.447584,2.023271,-0.441179,-0.415211,-0.037184,-0.033513,-0

In [3]:
X = df.iloc[:,1:]
X.head()

,PropertyID,censusblockgroup,Usecode,BedroomCnt,BathroomCnt,FinishedSquareFeet,GarageSquareFeet,LotSizeSquareFeet,StoryCnt,BuiltYear,ViewType,Latitude,Longitude,BGMedHomeValue,BGMedRent,BGMedYearBuilt,BGPctOwn,BGPctVacant,BGMedIncome,BGPctKids,BGMedAge,TransDate_4,TransDate_5,TransDate_6,TransDate_7,TransDate_8,TransDate_9,ZoneCodeCounty_A10,ZoneCodeCounty_A35,ZoneCodeCounty_AI1,ZoneCodeCounty_BO,ZoneCodeCounty_C1,ZoneCodeCounty_C140,ZoneCodeCounty_CB,ZoneCodeCounty_CBSO,ZoneCodeCounty_CM2,ZoneCodeCounty_DC,ZoneCodeCounty_DCE,ZoneCodeCounty_DUC,ZoneCodeCounty_F,...,ZoneCodeCounty_RS9600,ZoneCodeCounty_RSA 4,ZoneCodeCounty_RSA 6,ZoneCodeCounty_RSA 8,ZoneCodeCounty_RSE,ZoneCodeCounty_RSX 35,ZoneCodeCounty_RSX 7.2,ZoneCodeCounty_RSX 8.5,ZoneCodeCounty_SF 5000,ZoneCodeCounty_SF 7200,ZoneCodeCounty_SF 9600,ZoneCodeCounty_SFD,ZoneCodeCounty_SFE,ZoneCodeCounty_SFR 10.0,ZoneCodeCounty_SFS,ZoneCodeCounty_SFSL,ZoneCodeCounty_SR1,ZoneCodeCounty_SR3,ZoneCodeCounty_SR30,ZoneCodeCounty_SR4.5,ZoneCodeCounty_SR6,ZoneCodeCounty_SR8,ZoneCodeCounty_SVV,ZoneCodeCounty_T,ZoneCodeCounty_TC,ZoneCodeCounty_TC4,ZoneCodeCounty_UL15000,ZoneCodeCounty_UL7200,ZoneCodeCounty_UL9600,ZoneCodeCounty_UM2400,ZoneCodeCounty_UR,ZoneCodeCounty_URPSO,ZoneCodeCounty_UV,ZoneCodeCounty_UVEV,ZoneCodeCounty_WD II,Missing ViewType,Missing GarageSquareFeet,Missing BGMedYearBuilt,Missing BGMedRent,Missing BGMedHomeValue
0,-0.397302,0.0,0.0,0.633285,-0.375478,-0.320119,-0.049818,-0.197314,-1.012896,-0.371555,-0.326327,-1.610121,-0.100093,-1.829768,-0.992881,0.092411,-0.403853,0.462342,-1.433279,-1.193404,1.312359,-0.410486,2.115743,-0.447584,-0.494249,-0.441179,-0.415211,-0.037184,-0.033513,-0.00929,-0.00929,-0.022761,-0.013139,-0.016092,-0.00929,-0.00929,-0.00929,-0.00929,-0.013139,-0.026284,...,-0.034779,-0.044596,-0.138468,-0.037184,-0.00929,-0.018582,-0.102723,-0.00929,-0.48992,-0.204463,-0.055824,-0.026284,-0.024585,-0.00929,-0.068424,-0.053441,-0.022761,-0.050089,-0.00929,-0.084937,-0.177515,-0.052622,-0.00929,-0.00929,-0.016092,-0.00929,-0.016092,-0.085451,-0.013139,-0.013139,-0.062438,-0.087976,-0.077958,-0.058854,-0.013139,0.542108,-0.569910,-0.147578,-0.541975,-0.022761
1,-0.397299,0.0,0.0,-0.521924,-0.375478,-0.031915,-0.775699,-0.041773,-1.012896,-0.799322,-0.361972,-0.481783,-0.315709,-1.416003,-0.787479,-0.244938,-0.878712,-0.539812,-1.125734,0.083579,0.420317,-0.410486,-0.472647,-0.447584,-0.494249,2.266652,-0.415211,-0.037184,-0.033513,-0.00929,-0.00929,-0.022761,-0.013139,-0.016092,-0.00929,-0.00929,-0.00929,-0.00929,-0.013139,-0.026284,...,-0.034779,-0.044596,-0.138468,-0.037184,-0.00929,-0.018582,-0.102723,-0.00929,-0.48992,-0.204463,-0.055824,-0.026284,-0.024585,-0.00929,-0.068424,-0.053441,-0.022761,-0.050089,-0.00929,-0.084937,-0.177515,-0.052622,-0.00929,-0.00929,-0.016092,-0.00929,-0.016092,-0.085451,-0.013139,-0.013139,-0.062438,-0.087976,-0.077958,-0.058854,-0.013139,-1.844651,-0.569910,-0.147578,-0.541975,-0.022761
2,-0.397296,0.0,0.0,0.633285,-1.521526,-0.053264,0.449225,-0.220023,-1.012896,-0.728028,-0.374653,0.075045,-0.766053,-0.502016,-1.274357,-1.538110,-0.584218,-0.867046,-1.049419,-0.280155,0.137837,-0.410486,-0.472647,-0.447584,-0.494249,2.266652,-0.415211,-0.037184,-0.033513,-0.00929,-0.00929,-0.022761,-0.013139,-0.016092,-0.00929,-0.00929,-0.00929,-0.00929,-0.013139,-0.026284,...,-0.034779,-0.044596,-0.138468,-0.037184,-0.00929,-0.018582,-0.102723,-0.00929,-0.48992,4.890852,-0.055824,-0.026284,-0.024585,-0.00929,-0.068424,-0.053441,-0.022761,-0.050089,-0.00929,-0.084937,-0.177515,-0.052622,-0.00929,-0.00929,-0.016092,-0.00929,-0.016092,-0.085451,-0.013139,-0.013139,-0.062438,-0.087976,-0.077958,-0.058854,-0.013139,0.542108,-0.569910,-0.147578,-0.541975,-0.022761
3,-0.397295,0.0,0.0,0.633285,-0.088966,0.384379,-0.514406,-0.005720,-1.012896,-0.478497,-0.361972,-1.142897,-0.563556,-0.839428,-0.850874,0.204861,-1.030035,0.109543,-1.396183,-0.171960,0.033766,-0.410486,-0.472647,-0.447584,2.023271,-0.441179,-0.415211,-0.037184,-0.033513,-0.00

In [4]:
y = pd.DataFrame(df.iloc[:,0])
y.head()

,SaleDollarCnt
0,285000.0
1,309950.0
2,476000.0
3,324950.0
4,325000.0


In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
import numpy as np
knn = KNeighborsRegressor()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
knn.fit(X_train,y_train)
y_pred=knn.predict(X_test)
y_true=y_test
print(np.mean(np.abs((y_true - y_pred) / y_true)) )
output=pd.DataFrame(data=y_true)
output["KNN_pred"]=y_pred
output.head()

In [ ]:
output.shape

In [ ]:
def cross_validation_function(X,y,model,fold):
    size=int(len(X)/fold)
    l=0
    u=size
    scores=[]
    for f in range(fold):
        print("l =",l,"u =",u,"")
        dfxtest=X.iloc[l:u,:]
        dfytest=y.iloc[l:u,:]
        dfxtrain=X.drop(X.iloc[l:u].index,axis=0)
        dfytrain=y.drop(X.iloc[l:u].index,axis=0)
        model.fit(dfxtrain,dfytrain)
        y_pred=model.predict(dfxtest)
        y_true=dfytest
#         print(y_true)
#         print(y_pred)
        scores.append(np.mean(np.abs((y_true - y_pred) / y_true)))
        print("        FOLD ",f)
        print("        --> Score :", (np.mean(np.abs((y_true - y_pred) / y_true))))
        l=l+size
        u=u+size
    print("Final score is: ",np.mean(scores))
      
        

In [ ]:
from sklearn.model_selection import cross_val_score
for n in range(1,30):
    knn = KNeighborsRegressor(n_neighbors=n,weights="distance")
    print("N=",n)
    cross_validation_function(X,y,knn,10)


In [ ]:
from sklearn.model_selection import cross_val_score
for n in range(1,30):
    knn = KNeighborsRegressor(n_neighbors=n,weights="distance")
    print("N=",n)
    cross_validation_function(X,y,knn,5)


# Decision Tree Regression

In [ ]:
from sklearn.tree import DecisionTreeRegressor
import numpy as np

In [ ]:
def cross_validation_function2(X,y,model,fold):
    size=int(len(X)/fold)
    l=0
    u=size
    scores=[]
    for f in range(fold):
#         print("l =",l,"u =",u,"")
        dfxtest=X.iloc[l:u,:]
        dfytest=y.iloc[l:u,:]
        dfxtrain=X.drop(X.iloc[l:u].index,axis=0)
        dfytrain=y.drop(X.iloc[l:u].index,axis=0)
        model.fit(dfxtrain,dfytrain)
        y_pred=model.predict(dfxtest)
        y_true=pd.Series(dfytest.iloc[:,0])
#         pd.Series(y.iloc[:,0])
#         print(y_true)
#         print(y_pred)
        scores.append(np.mean(np.abs((y_true - y_pred) / y_true)))
        print("        FOLD ",f)
        print("        --> Score :", (np.mean(np.abs((y_true - y_pred) / y_true))))
        l=l+size
        u=u+size
    print("Final score is: ",np.mean(scores))
    return np.mean(scores)
      
        

In [ ]:
from sklearn.model_selection import cross_val_score
min=1
sc=[]
for d in range(1,30):
    tree = DecisionTreeRegressor(max_depth=d,criterion='mae')
    print("Depth =",d)
    newscore=cross_validation_function2(X,y,tree,5)
    sc.append(newscore)
    if(newscore<min):
        min=newscore
        print("Min CHANGED TO : ",min," at d =",d)
print(min(sc))


In [ ]:
from sklearn.model_selection import cross_val_score
sc=[]
min=1
for d in range(2,30):
    for nest in range(10,500,10):
        forest = RandomForestRegressor(max_depth=d, random_state=30,n_estimators=nest)
        print("N Estimators is : ",nest)
        print("Depth =",d)
        newscore=cross_validation_function3(X,y,forest,5)
        sc.append(newscore)
        if(newscore<min):
            min=newscore
            print("Min CHANGED TO : ",min," at d =",d," and n est = ",nest)
print(min(sc))
    # print(regr.feature_importances_)

Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
from sklearn.model_selection import cross_val_score
for d in range(1,30):
    for nest in range(10,200,10):
    forest = RandomForestRegressor(max_depth=d, random_state=30,n_estimators=100)
    print("Depth =",d)
    cross_validation_function(X,y,forest,5)
# print(regr.feature_importances_)

# Gradient Boosting Regressor

In [5]:
def cross_validation_function3(X,y,model,fold):
    size=int(len(X)/fold)
    l=0
    u=size
    scores=[]
    for f in range(fold):
#         print("l =",l,"u =",u,"")
        dfxtest=X.iloc[l:u,:]
        dfytest=y.iloc[l:u,:]
        dfxtrain=X.drop(X.iloc[l:u].index,axis=0)
        dfytrain=y.drop(X.iloc[l:u].index,axis=0)
        model.fit(dfxtrain,dfytrain.values.ravel())
        y_pred=model.predict(dfxtest)
        y_true=pd.Series(dfytest.iloc[:,0])
#         pd.Series(y.iloc[:,0])
#         print(y_true)
#         print(y_pred)
        scores.append(np.mean(np.abs((y_true - y_pred) / y_true)))
        print("        FOLD ",f)
        print("        --> Score :", (np.mean(np.abs((y_true - y_pred) / y_true))))
        l=l+size
        u=u+size
    print("Final score is: ",np.mean(scores))
    return np.mean(scores)
      
        

In [ ]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
sc=[]
min=1
for nest in range(10,500,10):
    for d in range(5,25):
        for l in np.arange(0.01,0.5,0.005):
            clf = ensemble.GradientBoostingRegressor(n_estimators= nest, max_depth=d,learning_rate=l , loss= 'ls',random_state=30)
            print("N Estimators is : ",nest)
            print("Depth =",d)
            newscore=cross_validation_function3(X,y,clf,5)
            sc.append(newscore)
            if(newscore<min):
                min=newscore
                print("Min CHANGED TO : ",min," at d =",d," and n est = ",nest,"l =",l)
print(min(sc))


 
